In [ ]:
import torchaudio


In [ ]:
def load_tags(file_path):
    tag_dict = {}
    with open(file_path, 'r') as file:
        for line in file:
            tag, code = line.strip().split()
            tag_dict[code] = tag
    return tag_dict

In [ ]:
tag_dict = load_tags('/home/ksingla/workspace/PromptingNemo/data_v2/synthetic/processed/alltags_uniq.txt')

In [ ]:
import nemo
import torch
import nemo.collections.asr as nemo_asr

map_location = torch.device('cpu')

asr_model = nemo_asr.models.ASRModel.restore_from("/external2/karan_exp/experiments/finetune-multidomain-alllang-100k-ctc/2024-07-18_00-21-12/checkpoints/finetune-multidomain-alllang-100k-ctc.nemo", map_location=map_location)
# Set the model to evaluation mode

asr_model.eval()

In [ ]:
# Define a function to map tags to words
def map_tags(word, tag_dict):
    parts = word.split('_') if '_' in word else word.split('-')
    mapped_parts = [tag_dict.get(part, part) for part in parts]
    return ' '.join(mapped_parts)

# Define a function to transcribe the audio
def transcribe(audio_file):
    # Load your 16k wav audio file
    audio, sample_rate = torchaudio.load(audio_file)

    if sample_rate != 16000:
        raise ValueError(f"Expected sample rate of 16000, but got {sample_rate}")

    # Perform transcription
    with torch.no_grad():
        transcription = asr_model.transcribe([audio_file])[0]

    # Map the tags to the corresponding words
    print(transcription)
    words = transcription.split()
    mapped_transcription = ' '.join([map_tags(word, tag_dict) for word in words])

    return mapped_transcription

In [ ]:
# Transcribe your audio file
audio_file = '/external2/datasets/LibriSpeech/test-converted-wav/1089-134686-0032.wav'
print(f"Transcription: {transcribe(audio_file)}")


In [ ]:
from IPython.display import Audio

#audio_file = '/external2/datasets/LibriSpeech/test-converted-wav/1089-134686-0023.wav'
Audio(audio_file)

In [ ]:
Transcription: DOMAIN SOCCER HIGHLIGHTS ENTITY PLAYER  was END the screment END of the ENTITY DEVICE ukaristu instuted underter the2o speies of bread and wine if jeeseuse cricet be presvanentt body and blood sol and divnity in the bread alonan and in theT813 wine a loan. INTENT ANALYSIS COMMENT

In [ ]:
from google.cloud import texttospeech
import os

# Set up your Google Cloud credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/ksingla/workspace/medical-ner/keys/google-tts-key.json"

def list_available_voices():
    # Initialize the Text-to-Speech client
    client = texttospeech.TextToSpeechClient()

    # Performs the list voices request
    response = client.list_voices()

    # Initialize a dictionary to store voices by language code
    voice_dict = {
        'EN': [], 'ES': [], 'FR': [], 'DE': [], 'IT': [], 'PT': [], 'NL': [], 'SV': []
    }

    # Define the language code mapping
    language_mapping = {
        'EN': 'en', 'ES': 'es', 'FR': 'fr', 'DE': 'de',
        'IT': 'it', 'PT': 'pt', 'NL': 'nl', 'SV': 'sv'
    }

    # Populate the dictionary with available voices
    for voice in response.voices:
        for language_code in voice.language_codes:
            for key, value in language_mapping.items():
                if language_code.startswith(value):
                    voice_dict[key].append(voice.name)

    return voice_dict

# List all available voices
voices = list_available_voices()

In [ ]:
voices

In [ ]:
from langdetect import detect, detect_langs

# Sample sentence
sentence = "나는 몸살이 나 오늘 아침 있어요"

# Detect language
language = detect(sentence)

# Detect language probabilities
languages = detect_langs(sentence)

print(f"The detected language is: {language}")
print(f"Language probabilities: {languages}")
